In [ ]:
import random
import pandas as pd
import numpy as np
import os
import json
import torch
import torch.nn as nn
from sklearn import preprocessing

from glob import glob

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
device = torch. device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
CFG = {
    'IMG_SIZE':384,
    'EPOCHS':50,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
# Label Encoding
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)

In [ ]:
def softmax_softvoting(save_file_name):
    all_files = glob(f'./csv_files_last/*.csv')
    preds = []
    softmax = nn.Softmax(dim = 1)
    for filename in all_files:
        df = pd.read_csv(filename,index_col=None,header=0)
        
        if len(preds) == 0:
            preds = df['artist']
            for i in range(len(preds)):
                x = [json.loads(preds[i])]
                x = softmax(torch.tensor(x))
                preds[i] = x.tolist()
            # print(preds.shape)
        else:
            for i in range(len(preds)):
                x = preds[i]
                x = torch.tensor(x)
                
                y = [json.loads(df['artist'][i])]
                y = softmax(torch.tensor(y))
                preds[i] = (x.add(y)).tolist()
                # if i == 0:
                #     print(x)
                #     print(y)
                #     print(x.add(y))
            # print(preds)
    
    submit = pd.read_csv('sample_submission.csv')
    
    preds = torch.tensor(preds).argmax(dim=-1)
    # preds = preds.argmax(dim=-1)
    preds = le.inverse_transform(preds)
    submit['artist'] = preds
    
    submit.to_csv(f'./{save_file_name}.csv', index = False)
    
def softvoting(save_file_name):
    all_files = glob(f'./csv_files_num/*.csv')
    preds = []
    for filename in all_files:
        df = pd.read_csv(filename,index_col=None,header=0)
        
        if len(preds) == 0:
            preds = df['artist']
            for i in range(len(preds)):
                x = [json.loads(preds[i])]
                x = torch.tensor(x)
                preds[i] = x.tolist()
            # print(preds.shape)
        else:
            for i in range(len(preds)):
                x = preds[i]
                # print(torch.tensor(x))
                x = torch.tensor(x)
                y = [json.loads(df['artist'][i])]
                y = torch.tensor(y)
                preds[i] = (x.add(y)).tolist()
                # if i == 0:
                #     print(x)
                #     print(y)
                #     print(x.add(y))
            # print(preds)
    
    submit = pd.read_csv('sample_submission.csv')
    
    preds = torch.tensor(preds).argmax(dim=-1)
    # preds = preds.argmax(dim=-1)
    preds = le.inverse_transform(preds)
    submit['artist'] = preds
    
    submit.to_csv(f'./{save_file_name}.csv', index = False)

In [ ]:
softmax_softvoting('ensemble_submit.csv')  # 제출용 csv 파일 생성